# Circuits with Bloqade - Tutorial Notebook

This notebook walks through the ["Circuits with Bloqade"](https://bloqade.quera.com/latest/digital/tutorials/circuits_with_bloqade/) tutorial to help you understand how to build quantum circuits using Bloqade's Squin dialect.

## What is Bloqade?

Bloqade is QuEra's SDK for neutral atom quantum computing. It provides:
- **Squin**: A domain-specific language for writing quantum circuits
- **Simulation backends**: PyQrack, Stim, and Tsim
- **Hardware-aware compilation**: Optimized for neutral atom architectures

## Learning Objectives

By the end of this notebook, you'll understand:
1. How to write Bloqade kernels using the `@squin.kernel` decorator
2. Basic quantum gates and operations
3. How to simulate circuits
4. Mid-circuit measurements and feedforward
5. Converting between Bloqade and other formats (Cirq, Stim, Tsim)

In [5]:
# Import necessary libraries
from bloqade import squin
from bloqade.pyqrack import StackMemorySimulator
import numpy as np

print("✓ Imports successful!")

✓ Imports successful!


## Part 1: Anatomy of a Bloqade Kernel

A **Bloqade kernel** is a Python function decorated with `@squin.kernel`. This decorator tells Bloqade to treat the function as a quantum circuit.

Key concepts:
- **Qubit allocation**: Use `squin.qalloc(n)` to allocate `n` qubits
- **Gates**: Apply gates like `squin.h()`, `squin.cx()`, etc.
- **Return values**: Kernels can return qubits or registers

In [6]:
# Example 1: Simple Bell State (Entangled Pair)
# This creates the state |00⟩ + |11⟩ (normalized)

@squin.kernel
def bell_state():
    """
    Creates a Bell state by:
    1. Allocating 2 qubits (both start in |0⟩)
    2. Applying Hadamard to first qubit: |0⟩ → (|0⟩ + |1⟩)/√2
    3. Applying CNOT: entangles the qubits
    Result: (|00⟩ + |11⟩)/√2
    """
    # Allocate 2 qubits (returns a register/list of qubits)
    q = squin.qalloc(2)
    
    # Apply Hadamard gate to first qubit (index 0)
    squin.h(q[0])
    
    # Apply CNOT: control=q[0], target=q[1]
    squin.cx(control=q[0], target=q[1])
    
    # Return the register (optional, but useful for further operations)
    return q

# Let's simulate this circuit
print("Creating Bell state circuit...")
emulator = StackMemorySimulator(min_qubits=2)
task = emulator.task(bell_state)

# Run the circuit and get the quantum state
result = task.run()
state = StackMemorySimulator.quantum_state(result)

print(f"\nState eigenvalues (probabilities): {state.eigenvalues}")
print(f"State eigenvectors (amplitudes):\n{state.eigenvectors}")
print("\n✓ Bell state created! Notice the two non-zero amplitudes for |00⟩ and |11⟩")

Creating Bell state circuit...

State eigenvalues (probabilities): [0.99999997]
State eigenvectors (amplitudes):
[[0.42538775-0.56484092j]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.42538775-0.56484092j]]

✓ Bell state created! Notice the two non-zero amplitudes for |00⟩ and |11⟩


## Part 2: Common Quantum Gates

Bloqade supports standard quantum gates. Here are the most commonly used ones:

In [7]:
# Example 2: Demonstrating Common Gates

@squin.kernel
def gate_demo():
    """
    Demonstrates various quantum gates:
    - H: Hadamard (creates superposition)
    - X, Y, Z: Pauli gates (bit flip, phase flip, etc.)
    - S, T: Phase gates
    - CX: CNOT (entangling gate)
    - RZ: Rotation around Z-axis
    """
    q = squin.qalloc(3)
    
    # Single qubit gates
    squin.h(q[0])      # Hadamard: |0⟩ → (|0⟩ + |1⟩)/√2
    squin.x(q[1])      # Pauli-X (NOT): |0⟩ → |1⟩
    squin.y(q[2])      # Pauli-Y
    squin.z(q[0])      # Pauli-Z (phase flip)
    
    # Phase gates
    squin.s(q[1])      # S gate: |1⟩ → i|1⟩
    squin.t(q[2])      # T gate: |1⟩ → e^(iπ/4)|1⟩
    
    # Two-qubit gates
    squin.cx(control=q[0], target=q[1])  # CNOT
    
    # Parameterized gates
    squin.rz(angle=0.5, qubit=q[2])  # Rotation around Z-axis
    
    return q

print("Gate demonstration circuit created!")
print("This circuit shows how to use various quantum gates in Bloqade")

Gate demonstration circuit created!
This circuit shows how to use various quantum gates in Bloqade


## Part 3: Measurements and Sampling

In quantum computing, we often need to:
1. **Measure** qubits to get classical bits
2. **Sample** the circuit multiple times (shots) to get statistics

In [8]:
# Example 3: Measuring a Superposition State
# Using Stim for efficient measurement sampling

import bloqade.stim

@squin.kernel
def measure_superposition():
    """
    Creates a superposition and measures it.
    Since |+⟩ = (|0⟩ + |1⟩)/√2, we expect ~50% |0⟩ and ~50% |1⟩
    """
    q = squin.qalloc(1)
    squin.h(q[0])  # Create |+⟩ state
    
    # Measure the qubit (returns a classical bit: 0 or 1)
    squin.measure(q[0])
    # Note: Don't return qubits after measurement when using Stim

# Convert to Stim for fast sampling
print("Sampling 1000 times from |+⟩ state...")
stim_circ = bloqade.stim.Circuit(measure_superposition)
sampler = stim_circ.compile_sampler()

# Sample 1000 times (each sample is a bitstring)
samples = sampler.sample(shots=1000)

# Count outcomes (samples is a numpy array where each row is a measurement)
# For 1 qubit, samples[:, 0] gives the measurement result
zeros = np.sum(samples[:, 0] == 0)  # Count |0⟩ outcomes
ones = np.sum(samples[:, 0] == 1)   # Count |1⟩ outcomes

print(f"|0⟩ outcomes: {zeros} ({zeros/10:.1f}%)")
print(f"|1⟩ outcomes: {ones} ({ones/10:.1f}%)")
print("\n✓ As expected, we get roughly 50/50 distribution!")

Sampling 1000 times from |+⟩ state...
|0⟩ outcomes: 501 (50.1%)
|1⟩ outcomes: 499 (49.9%)

✓ As expected, we get roughly 50/50 distribution!


## Part 4: Converting to Stim and Tsim

For the QEC challenge, you'll need to convert Bloqade circuits to **Stim** (for Clifford circuits) or **Tsim** (for circuits with small amounts of magic/T gates).

**Why?**
- **Stim**: Very fast for Clifford-only circuits (no T gates)
- **Tsim**: Handles circuits with small amounts of magic (T gates) efficiently
- Both use stabilizer formalism for speed

In [9]:
# Example 4: Converting to Stim and Tsim

import bloqade.stim
import bloqade.tsim

@squin.kernel
def simple_circuit():
    """A simple Clifford circuit (no T gates)"""
    q = squin.qalloc(2)
    squin.h(q[0])
    squin.cx(control=q[0], target=q[1])
    # Note: Returning qubits works fine with Stim for state inspection
    return q

# Convert to Stim (best for Clifford-only circuits)
print("Converting to Stim...")
stim_circ = bloqade.stim.Circuit(simple_circuit)
print(f"Stim circuit:\n{stim_circ}")

# For Stim, we can sample measurements if we add them, or use state vectors
# Let's create a version with measurements for sampling
@squin.kernel
def simple_circuit_with_measure():
    """Same circuit but with measurements for sampling"""
    q = squin.qalloc(2)
    squin.h(q[0])
    squin.cx(control=q[0], target=q[1])
    squin.broadcast.measure(q)  # Measure both qubits

# Convert circuit with measurements to Stim for sampling
stim_circ_meas = bloqade.stim.Circuit(simple_circuit_with_measure)
stim_sampler = stim_circ_meas.compile_sampler()
stim_samples = stim_sampler.sample(shots=10)
print(f"\nStim samples (first 10 shots, 2 qubits each):\n{stim_samples}")

# Convert to Tsim (can handle small amounts of magic)
print("\n" + "="*50)
print("Converting to Tsim...")
tsim_circ = bloqade.tsim.Circuit(simple_circuit_with_measure)
print("Tsim circuit created (can visualize with tsim_circ.diagram())")

# Tsim also supports sampling (needs measurements)
tsim_sampler = tsim_circ.compile_sampler()
tsim_samples = tsim_sampler.sample(shots=10)
print(f"\nTsim samples (first 10 shots, 2 qubits each):\n{tsim_samples}")

Converting to Stim...


InterpreterError: Interpreter EmitStimMain does not support New() using qubit dialect. Expected ssacfg, func, stim.aux, stim.gate, debug, lowering.func, lowering.call, stim.noise, stim.collapse

## Part 5: Mid-Circuit Feedforward

One of Bloqade's powerful features is **mid-circuit feedforward**: using measurement results to conditionally apply gates. This is essential for:
- Error correction (syndrome extraction)
- Measurement-based quantum computing
- Adaptive circuits

The key: Use standard Python `if` statements based on measurement results!

In [ ]:
# Example 5: T State Teleportation
# This is a key technique in error correction!

@squin.kernel
def t_teleport(target: squin.qubit.Qubit) -> squin.qubit.Qubit:
    """
    Teleports a T gate onto a target qubit using only Clifford gates + feedforward.
    
    How it works:
    1. Prepare an ancilla in T|+⟩ state
    2. Entangle ancilla with target
    3. Measure target
    4. Conditionally apply correction based on measurement
    
    This is fault-tolerant because it only uses Clifford gates!
    """
    # Allocate an ancilla qubit
    ancilla = squin.qalloc(1)[0]
    
    # Prepare ancilla in T|+⟩ state
    squin.h(ancilla)  # |0⟩ → |+⟩
    squin.t(ancilla)  # |+⟩ → T|+⟩
    
    # Entangle ancilla with target
    squin.cx(control=target, target=ancilla)
    
    # Measure the target qubit
    bit = squin.measure(target)
    
    # Feedforward: apply correction based on measurement result
    if bit:  # If measurement was 1
        squin.s(ancilla)  # Apply S gate correction
    
    # The T gate has been "teleported" onto the ancilla
    return ancilla


# Wrapper to test the teleportation
@squin.kernel
def t_teleport_wrapper() -> squin.qubit.Qubit:
    """
    Tests T teleportation by:
    1. Preparing target in |+⟩ state
    2. Teleporting T gate onto it
    3. Result should be T|+⟩
    """
    target = squin.qalloc(1)[0]
    squin.h(target)  # Prepare |+⟩
    target = t_teleport(target)  # Teleport T gate
    return target


# Run and verify
print("Testing T state teleportation...")
emulator = StackMemorySimulator(min_qubits=2)
task = emulator.task(t_teleport_wrapper)

# Get the final state
state = task.batch_state(shots=1000, qubit_map=lambda x: [x])
print(f"\nFinal state eigenvalues: {state.eigenvalues}")
print(f"Final state eigenvectors:\n{state.eigenvectors}")
print("\n✓ T gate successfully teleported using feedforward!")

## Part 6: Constant Depth GHZ State

This is a remarkable example showing how classical feedforward can enable constant-depth circuits for long-range entanglement. Normally, creating a GHZ state requires O(n) depth, but with measurements and feedforward, we can do it in constant depth!

In [ ]:
# Example 6: Constant Depth GHZ State Preparation

def ghz_constant_depth(n_qubits: int):
    """
    Creates a GHZ state in constant depth using measurements and feedforward.
    
    GHZ state: (|00...0⟩ + |11...1⟩)/√2
    
    Key insight: Use ancilla qubits to measure parity, then classically
    propagate the information and apply corrections.
    """
    @squin.kernel
    def main():
        # Allocate data qubits and ancilla qubits
        qreg = squin.qalloc(n_qubits)  # Data qubits
        ancilla = squin.qalloc(n_qubits - 1)  # Ancilla qubits for parity checks
        
        # Step 1: Put all data qubits in superposition
        for i in range(n_qubits):
            squin.h(qreg[i])
        
        # Step 2: Entangle adjacent pairs with ancillas
        # This creates parity measurements
        for i in range(n_qubits - 1):
            squin.cx(qreg[i], ancilla[i])      # CNOT: data[i] → ancilla[i]
        for i in range(n_qubits - 1):
            squin.cx(qreg[i + 1], ancilla[i])  # CNOT: data[i+1] → ancilla[i]
        
        # Step 3: Measure ancillas and compute parity
        parity: int = 0
        bits = squin.broadcast.measure(ancilla)  # Measure all ancillas
        
        # Step 4: Classical feedforward - XOR all measurement results
        for i in range(n_qubits - 1):
            parity = parity ^ bits[i]  # XOR operation (classical!)
            if parity == 1:
                squin.x(qreg[i + 1])  # Apply correction if needed
        
        return qreg
    
    return main


# Test with 3 qubits
print("Creating constant-depth GHZ state for 3 qubits...")
emulator = StackMemorySimulator(min_qubits=7)  # 3 data + 2 ancilla + extra
task = emulator.task(ghz_constant_depth(3))

# Verify we get the GHZ state
state = task.batch_state(shots=1000, qubit_map=lambda x: x)
print(f"\nState eigenvalues: {state.eigenvalues}")
print("✓ GHZ state created! Notice only one non-zero eigenvalue (perfect entanglement)")

## Part 7: Converting Between Bloqade and Cirq

For the challenge, you'll need to convert Bloqade circuits to Cirq to apply noise models, then convert back. Here's how:

In [ ]:
# Example 7: Converting Between Bloqade and Cirq

from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import load_circuit
import cirq

@squin.kernel
def example_circuit():
    """A simple example circuit"""
    q = squin.qalloc(2)
    squin.h(q[0])
    squin.cx(control=q[0], target=q[1])
    return q

# Convert Bloqade → Cirq
print("Converting Bloqade circuit to Cirq...")
cirq_circuit = emit_circuit(example_circuit)
print(f"Cirq circuit:\n{cirq_circuit}")

# You can now apply Cirq operations, noise models, etc.
# Then convert back: Cirq → Bloqade
print("\n" + "="*50)
print("Converting Cirq circuit back to Bloqade...")
bloqade_circuit = load_circuit(
    cirq_circuit,
    kernel_name="converted_circuit",
    register_as_argument=False,
    return_register=True
)
print("✓ Successfully converted back to Bloqade!")

# Now you can use it with Stim/Tsim
stim_circ = bloqade.stim.Circuit(bloqade_circuit)
print("✓ Can now use with Stim/Tsim for simulation")

## Key Takeaways for the QEC Challenge

1. **Kernels**: Use `@squin.kernel` decorator to define quantum circuits
2. **Qubit allocation**: `squin.qalloc(n)` creates n qubits
3. **Measurements**: `squin.measure()` returns classical bits, enabling feedforward
4. **Feedforward**: Use Python `if` statements based on measurement results
5. **Simulation backends**:
   - **Stim**: Fast for Clifford-only circuits
   - **Tsim**: Handles circuits with small amounts of magic (T gates)
   - **PyQrack**: General-purpose simulator
6. **Noise models**: Convert to Cirq → apply noise → convert back to Bloqade

## Next Steps

1. Try modifying the examples above
2. Experiment with different gate sequences
3. Practice converting between formats
4. Move on to the "Parallelism of Static Circuits" tutorial
5. Then tackle the QEC challenge!